# Zurich Instruments LabOne Python API Example
# Continuous Data Acquisition

Python API Example for the Data Acquisition Module. This example demonstrates
how to record data from an instrument continuously (without triggering).
Record data continuously in 0.2 s chunks for 5 seconds using the Data
Acquisition Module.

> Note:
>
> This example does not perform any device configuration. If the streaming nodes
> corresponding to the signal_paths are not enabled, no data will be recorded.

Requirements:
* LabOne Version >= 20.02
* Instruments:
    1 x Instrument with demodulators
* Signal output 1 connected to signal input 1 with a BNC cable.
---

In [ ]:
import numpy as np
import time
import zhinst.core
%matplotlib notebook
import matplotlib.pyplot as plt

Set up the connection. The connection is always through a session to a
Data Server. The Data Server then connects to the device.

The LabOne Data Server needs to run within the network, either on localhost when
starting LabOne on your local computer or a remote server. (The MFLI/IA devices
have a Data Server running on the device itself, please see the
[user manual](https://docs.zhinst.com/mfli_user_manual/getting_started/device_connectivity_instrument.html)
for detailed explanation.)

In [ ]:
device_id = "dev2345" # Device serial number available on its rear panel.
interface = "1GbE" # For Ethernet connection or when MFLI/MFIA is connected to a remote Data Server.
#interface = "USB" # For all instruments connected to the host computer via USB except MFLI/MFIA.
#interface = "PCIe" # For MFLI/MFIA devices in case the Data Server runs on the device.

server_host = "localhost"
server_port = 8004
#server_port = 8005 # Default port for HF2LI.
api_level = 6 # Maximum API level supported for all instruments except HF2LI.
#api_level = 1 # Maximum API level supported for HF2LI.

# Create an API session to the Data Server.
daq = zhinst.core.ziDAQServer(server_host, server_port, api_level)
# Establish a connection between Data Server and Device.
daq.connectDevice(device_id, interface)

Enable Demodulator 0

In [ ]:
daq.set(f"/{device_id}/demods/0/enable", 1)

Define signal paths that should be recorded in the module.

A full list of possible signals can be found in the
[user manual](https://docs.zhinst.com/labone_programming_manual/data_acquisition_module.html#pm.core.modules.daq.signalsubscription).

In [ ]:

demod_path = f"/{device_id}/demods/0/sample"
signal_paths = []
signal_paths.append(demod_path + ".x")  # The demodulator X output.
signal_paths.append(demod_path + ".y")  # The demodulator Y output.

Defined the total time we would like to record data for and its sampling rate.

In [ ]:

total_duration = 5 # Time in seconds for the aquisition.
module_sampling_rate = 30000  # Number of points/second.
burst_duration = 0.2  # Time in seconds for each data burst/segment.
num_cols = int(np.ceil(module_sampling_rate * burst_duration))
num_bursts = int(np.ceil(total_duration / burst_duration))

Create an instance of the Data Acquisition Module

In [ ]:

daq_module = daq.dataAcquisitionModule()

Set the device that will be used for the trigger.

**This parameter must be set** even if the module runs in continuos mode.

In [ ]:
daq_module.set("device", device_id)

Configure the DAQ module.

In [ ]:
# Specify continuous acquisition. (continuous = 0)
daq_module.set("type", "continuous")
# 'grid/mode' - Specify the interpolation method of the returned data samples.
# (use ``daq_module.help("grid/mode")`` to se the available options)
# (linear = 2)
daq_module.set("grid/mode", "linear")
# 'count' - Specify the number of bursts of data the
#   module should return (if endless=0). The
#   total duration of data returned by the module will be
#   count*duration.
daq_module.set("count", num_bursts)
# 'duration' - Burst duration in seconds.
#   If the data is interpolated linearly or using nearest neighbor, specify
#   the duration of each burst of data that is returned by the DAQ Module.
daq_module.set("duration", burst_duration)
# 'grid/cols' - The number of points within each duration.
#   This parameter specifies the number of points to return within each
#   burst (duration seconds worth of data) that is
#   returned by the DAQ Module.
daq_module.set("grid/cols", num_cols)

**Optionaly** setup the storage of the data to a file

In [ ]:
filename = "Example_Data_Acquisition_Continous"
# 'save/fileformat' - The file format to use for the saved data.
#    0 - Matlab
#    1 - CSV
daq_module.set("save/fileformat", "csv")
# 'save/filename' - Each file will be saved to a
# new directory in the Zurich Instruments user directory with the name
# filename_NNN/filename_NNN/
daq_module.set("save/filename", filename)
# 'save/saveonread' - Automatically save the data
# to file each time read() is called.
daq_module.set("save/saveonread", True)

Subscribe to all signal paths.

In [ ]:
data = {}
for signal_path in signal_paths:
    print("Subscribing to ", signal_path)
    daq_module.subscribe(signal_path)
    data[signal_path] = []

Continuously acquire (and plot) the data for the subscribed signals for the
specified duration.

> Warning:
>
> This examples stores all the acquired data in the `data` dict - remove this
> continuous storing in `read_new_data` before increasing the size of
> total_duration!

In [ ]:
do_plot = True

clockbase = float(daq.getInt(f"/{device_id}/clockbase"))
if do_plot:
    timestamp0 = None
    max_value = None
    min_value = None
    fig, axis = plt.subplots()
    axis.set_xlabel("Time ($s$)")
    axis.set_ylabel("Subscribed signals")
    axis.set_xlim([0, total_duration])
    lines = [axis.plot([], [], label=path)[0] for path in signal_paths]
    axis.legend()
    axis.set_title("Continuous Data Acquisition")
    plt.ion()


def process_data(raw_data):
    global timestamp0, lines, max_value, min_value
    for i, signal_path in enumerate(signal_paths):
        # Loop over all the bursts for the subscribed signal. More than
        # one burst may be returned at a time, in particular if we call
        # read() less frequently than the burst_duration.
        for signal_burst in raw_data.get(signal_path.lower(), []):
            # Convert from device ticks to time in seconds.
            value = signal_burst["value"][0, :]
            data[signal_path].append(value)
            if do_plot:
                max_value = max(max_value, max(value)) if max_value else max(value)
                min_value = min(min_value, min(value)) if min_value else min(value)
                axis.set_ylim(min_value, max_value)
                timestamp0 = (
                    timestamp0 if timestamp0 else signal_burst["timestamp"][0, 0]
                )
                t = (signal_burst["timestamp"][0, :] - timestamp0) / clockbase
                lines[i].set_data(
                    np.concatenate((lines[i].get_xdata(), t), axis=0),
                    np.concatenate((lines[i].get_ydata(), value), axis=0),
                )
    if do_plot:
        fig.canvas.draw()

# Start recording data.
daq_module.execute()
# Record data in a loop with timeout.
timeout = 1.5 * total_duration
start = time.time()

while not daq_module.finished():
    t0_loop = time.time()
    if time.time() - start > timeout:
        raise Exception(
            f"Timeout after {timeout} s - recording not complete."
            "Are the streaming nodes enabled?"
            "Has a valid signal_path been specified?"
        )
    raw_data = daq_module.read(True)
    process_data(raw_data)
    time.sleep(max(0, burst_duration - (time.time() - t0_loop)))
# There may be new data between the last read() and calling finished().
raw_data = daq_module.read(True)
process_data(raw_data)